In [ ]:
%load_ext autoreload
%autoreload 2

# Cross Validation Functions

> Fill in a module description here

In [ ]:
#| default_exp crossValidation

In [ ]:
#| hide
from nbdev.showdoc import *
# from nbdev.qmd import *

In [ ]:
#| export
from __future__ import annotations
from fastcore.docments import *
from fastcore.test import *
from fastcore.utils import *

import pandas as pd
import numpy as np

from joblib import Parallel, delayed, dump, load
from collections import defaultdict
import copy

from sklearn.model_selection import ParameterGrid, ParameterSampler
from sklearn.base import clone

from dddex.wSAA import SampleAverageApproximation

## Cross Validation - General Quantile Predictors

In [ ]:
#| export

class QuantileCrossValidation:
    """
    Class to efficiently tune the `binSize` parameter of all Level-Set based models.
    """
    
    def __init__(self,
                 # An object with a `predict` method that must (!) have an argument called `probs`
                 # that specifies which quantiles to predict. Further, `quantileEstimator` needs
                 # a `set_params` and `fit` method.
                 quantileEstimator, 
                 cvFolds, # An iterable yielding (train, test) splits as arrays of indices.
                 parameterGrid: dict,
                 probs: list=[i / 100 for i in range(1, 100, 1)], # list or array of floats between 0 and 1. p-quantiles being predicted to evaluate performance of LSF.
                 # If True, for each p-quantile a fitted LSF with best binSize to predict it is returned. 
                 # Otherwise only one LSF is returned that is best over all probs.
                 refitPerProb: bool=False, 
                 n_jobs: int=None, # number of folds being computed in parallel.
                 ):
        
        # CHECKS  
        # if not isinstance(estimatorLSx, (BaseLSx)):
        #     raise ValueError("'estimatorLSx' has to be a 'LevelSetKDEx', 'LevelSetKDEx_NN_new' or a 'LevelSetKDEx_kNN' object!")               
            
        if np.any(np.array(probs) > 1) or np.any(np.array(probs) < 0): 
            raise ValueError("probs must only contain numbers between 0 and 1!") 
        
        #---
        
        self.quantileEstimator = copy.deepcopy(quantileEstimator)
        self.parameterGrid = ParameterGrid(parameterGrid)
        self.cvFolds = cvFolds
        self.probs = probs
        self.refitPerProb = refitPerProb
        self.n_jobs = n_jobs
        
        self.bestParams = None
        self.bestParams_perProb = None
        self.bestEstimator = None
        self.bestEstimator_perProb = None
        self.cvResults = None
        self.cvResults_raw = None
        

In [ ]:
#| export

@patch
def fit(self: QuantileCrossValidation, 
        X: np.ndarray, # Feature matrix (has to work with the folds specified via `cvFolds`)
        y: np.ndarray, # Target values (has to work with the folds specified via `cvFolds`)
        ): 
    
    # Making sure that X and y are arrays to ensure correct subsetting via implicit indices.
    X = np.array(X)
    y = np.array(y)
    
    # scoresPerFold = Parallel(n_jobs = self.n_jobs)(delayed(getFoldScore)(estimatorLSx = copy.deepcopy(self.quantileEstimator),
    #                                                                      parameterGrid = self.parameterGrid,
    #                                                                      cvFold = cvFold,
    #                                                                      probs = self.probs,
    #                                                                      X = X,
    #                                                                      y = y) for cvFold in self.cvFolds)
    
    scoresPerFold = [getFoldScore(quantileEstimator = copy.deepcopy(self.quantileEstimator),
                                  parameterGrid = self.parameterGrid,
                                  cvFold = cvFold,
                                  probs = self.probs,
                                  y = y,
                                  X = X) for cvFold in self.cvFolds]

    self.cvResults_raw = scoresPerFold
    meanCostsDf = sum(scoresPerFold) / len(scoresPerFold)
    self.cvResults = meanCostsDf
    
    #---
    
    # BEST PARAMETER SETTINGS OVER ALL PROBS
    meanCostsPerParam = meanCostsDf.mean(axis = 1)
    paramsBest = meanCostsPerParam.index[np.argmin(meanCostsPerParam)]
    paramsBest = dict(zip(meanCostsPerParam.index.names, paramsBest))
    self.bestParams = paramsBest
    
    #---
    
    # BEST PARAMETER SETTINGS PER PROB
    paramsBestPerProbSeries = meanCostsDf.idxmin(axis = 0)
    paramsBestPerProb = {prob: dict(zip(meanCostsDf.index.names, paramsBestPerProbSeries.loc[prob])) for prob in self.probs}
    self.bestParams_perProb = paramsBestPerProb
    
    #---
    
    # REFITTING ESTIMATORS
    if self.refitPerProb:      
        
        # GET UNIQUE PARAMETER COMBS OF LSx THAT ARE BEST FOR ANY PROB
        paramsUnique = pd.DataFrame(paramsBestPerProb.values()).drop_duplicates().to_dict(orient = 'records')
        estimatorDict = dict()
        
        for params in paramsUnique:
            quantileEstimator = copy.deepcopy(self.quantileEstimator)
            quantileEstimator.set_params(**params)
            
            quantileEstimator.fit(X = X, y = y)
            estimatorDict[tuple(params.values())] = quantileEstimator
        
        #---
        
        # DICTIONARY OF THE BEST LSx-ESTIMATORS PER PROB
        bestEstimatorPerProb = {prob: estimatorDict[tuple(paramsBestPerProb[prob].values())] for prob in self.probs}
        self.bestEstimator_perProb = bestEstimatorPerProb
        
    #---
    
    quantileEstimator = copy.deepcopy(self.quantileEstimator)
    quantileEstimator.set_params(**paramsBest)
    quantileEstimator.fit(X = X, y = y)

    self.bestEstimator = quantileEstimator

In [ ]:
# show_doc(CrossValidationLSx.fit)

#### Scores for Single Fold

In [ ]:
#| export

# This function evaluates the newsvendor performance for different bin sizes for one specific fold.
# The considered bin sizes

def getFoldScore(quantileEstimator, parameterGrid, cvFold, probs, X, y):
    
    indicesTrain = cvFold[0]
    indicesTest = cvFold[1]
    
    yTrainFold = y[indicesTrain]
    XTrainFold = X[indicesTrain]
    
    yTestFold = y[indicesTest]
    XTestFold = X[indicesTest]
    
    #---

    SAA_fold = SampleAverageApproximation()
    SAA_fold.fit(y = yTrainFold)

    # By setting 'X = None', the SAA results are only computed for a single observation (they are independent of X anyway).
    # In order to receive the final dataframe of SAA results, we simply duplicate this single row as many times as needed.
    quantilesDictSAAOneOb = SAA_fold.predict(X = None, probs = probs, outputAsDf = False)
    quantilesDictSAA = {prob: np.repeat(quantile, len(XTestFold)) for prob, quantile in quantilesDictSAAOneOb.items()}

    #---
    
    # Necessary to ensure compatability with wSAA-models etc.
    try:
        quantileEstimator.refitPointEstimator(X = XTrainFold, 
                                              y = yTrainFold)
    except:
        pass

    costsPerParam = defaultdict(dict)

    for params in parameterGrid:

        quantileEstimator.set_params(**params)

        quantileEstimator.fit(X = XTrainFold,
                              y = yTrainFold)

        quantilesDict = quantileEstimator.predict(X = XTestFold,
                                                  probs = probs,
                                                  outputAsDf = False)

        costsDict = dict()
        for prob in probs:            
            costsDict[prob] = getCostRatio(decisions = quantilesDict[prob], 
                                           decisionsSAA = quantilesDictSAA[prob], 
                                           yTest = yTestFold, 
                                           prob = prob)

        costsPerParam[tuple(params.values())] = costsDict

    #---

    costsDf = pd.DataFrame.from_dict(costsPerParam, orient = 'index')
    costsDf.index.names = list(params.keys())
    
    return costsDf

## Cross Validation - LSx + Point Predictor

In [ ]:
#| export

class CrossValidationLSx_combined:
    """
    Class to efficiently tune the `binSize` parameter of all Level-Set based models.
    """
    
    def __init__(self,
                 estimatorLSx, # A Level-Set based model.
                 cvFolds, # An iterable yielding (train, test) splits as arrays of indices.
                 parameterGridLSx: dict,
                 parameterGridEstimator: dict,
                 probs: list=[i / 100 for i in range(1, 100, 1)], # list or array of floats between 0 and 1. p-quantiles being predicted to evaluate performance of LSF.
                 # If True, for each p-quantile a fitted LSF with best binSize to predict it is returned. 
                 # Otherwise only one LSF is returned that is best over all probs.
                 refitPerProb: bool=False, 
                 n_jobs: int=None, # number of folds being computed in parallel.
                 ):
        
        # CHECKS  
        if not isinstance(estimatorLSx, (BaseLSx)):
            raise ValueError("'estimatorLSx' has to be a 'LevelSetKDEx', 'LevelSetKDEx_NN_new' or a 'LevelSetKDEx_kNN' object!")               
            
        if np.any(np.array(probs) > 1) or np.any(np.array(probs) < 0): 
            raise ValueError("probs must only contain numbers between 0 and 1!") 
        
        #---
        
        self.estimatorLSx = copy.deepcopy(estimatorLSx)
        self.parameterGridLSx = ParameterGrid(parameterGridLSx)
        self.parameterGridEstimator = ParameterGrid(parameterGridEstimator)
        self.cvFolds = cvFolds
        self.probs = probs
        self.refitPerProb = refitPerProb
        self.n_jobs = n_jobs
        
        self.bestParams = None
        self.bestParams_perProb = None
        self.bestEstimatorLSx = None
        self.bestEstimatorLSx_perProb = None
        self.cvResults = None
        self.cvResults_raw = None
        

In [ ]:
#| export

@patch
def fit(self: CrossValidationLSx_combined, 
        X: np.ndarray, # Feature matrix (has to work with the folds specified via `cvFolds`)
        y: np.ndarray, # Target values (has to work with the folds specified via `cvFolds`)
        ): 
    
    # Making sure that X and y are arrays to ensure correct subsetting via implicit indices.
    X = np.array(X)
    y = np.array(y)
    
    # scoresPerFold = Parallel(n_jobs = self.n_jobs)(delayed(getFoldScore_combined)(cvFold = cvFold,
    #                                                                               binSizeGrid = self.binSizeGrid,
    #                                                                               probs = self.probs,
    #                                                                               estimatorLSx = copy.deepcopy(self.estimatorLSx),
    #                                                                               y = y,
    #                                                                               X = X) for cvFold in self.cvFolds)
    
    scoresPerFold = [getFoldScore_combined(estimatorLSx = copy.deepcopy(self.estimatorLSx),
                                           parameterGridLSx = self.parameterGridLSx,
                                           parameterGridEstimator = self.parameterGridEstimator,
                                           cvFold = cvFold,
                                           probs = self.probs,
                                           y = y,
                                           X = X) for cvFold in self.cvFolds]

    self.cvResults_raw = scoresPerFold
    meanCostsDf = sum(scoresPerFold) / len(scoresPerFold)
    self.cvResults = meanCostsDf
    
    #---
    
    # BEST PARAMETER SETTINGS OVER ALL PROBS
    meanCostsPerParam = meanCostsDf.mean(axis = 1)
    paramsBest = meanCostsPerParam.index[np.argmin(meanCostsPerParam)]
    paramsBest = dict(zip(meanCostsPerParam.index.names, paramsBest))
    
    paramsLSxNames = self.estimatorLSx._get_param_names()
    paramsLSxBest = {paramName: value for paramName, value in paramsBest.items() if paramName in paramsLSxNames}
    paramsEstimatorBest = {paramName: value for paramName, value in paramsBest.items() if not paramName in paramsLSxNames}
    
    self.bestParams = paramsBest
    self.bestParamsLSx = paramsLSxBest
    self.bestParamsEstimator = paramsEstimatorBest
    
    #---
    
    # BEST PARAMETER SETTINGS PER PROB
    paramsBestPerProbSeries = meanCostsDf.idxmin(axis = 0)
    paramsBestPerProb = {prob: dict(zip(meanCostsDf.index.names, paramsBestPerProbSeries.loc[prob])) for prob in self.probs}
    
    paramsLSxBestPerProb = {prob: {paramName: value for paramName, value in paramsBestPerProb[prob].items() 
                                   if paramName in paramsLSxNames} for prob in self.probs}
    paramsEstimatorBestPerProb = {prob: {paramName: value for paramName, value in paramsBestPerProb[prob].items() 
                                   if not paramName in paramsLSxNames} for prob in self.probs}
    
    self.bestParams_perProb = paramsBestPerProb
    self.bestParamsLSx_perProb = paramsBestPerProb
    self.bestParamsEstimator_perProb = paramsBestPerProb
    
    #---
    
    # REFITTING ESTIMATORS
    if self.refitPerProb:
        
        # GET UNIQUE PARAMETER COMBS OF ESTIMATOR THAT ARE BEST FOR ANY PROB
        paramsEstimatorUnique = pd.DataFrame(paramsEstimatorBestPerProb.values()).drop_duplicates().to_dict(orient = 'records')
        estimatorDict = dict()
        
        for params in paramsEstimatorUnique:
            estimator = clone(self.estimatorLSx.estimator)
            estimator.set_params(**params)
            estimator.fit(X = X, y = y)
            estimatorDict[tuple(params.values())] = estimator
        
        #---
        
        # GET UNIQUE PARAMETER COMBS OF LSx THAT ARE BEST FOR ANY PROB
        paramsUnique = pd.DataFrame(paramsBestPerProb.values()).drop_duplicates().to_dict(orient = 'records')
        
        estimatorLSxDict = dict()
        for params in paramsUnique:
            paramsLSx = {paramName: value for paramName, value in params.items() if paramName in paramsLSxNames}
            paramsEstimatorTuple = tuple(value for paramName, value in params.items() if not paramName in paramsLSxNames)
            
            estimator = estimatorDict[paramsEstimatorTuple]
            estimatorLSx = copy.deepcopy(self.estimatorLSx)
            
            estimatorLSx.set_params(**paramsLSx, 
                                    estimator = estimator)
            
            estimatorLSx.fit(X = X, y = y)
            estimatorLSxDict[tuple(params.values())] = estimatorLSx
        
        #---
        
        # DICTIONARY OF THE BEST LSx-ESTIMATORS PER PROB
        bestEstimatorPerProb = {prob: estimatorLSxDict[tuple(paramsBestPerProb[prob].values())] for prob in self.probs}
        self.bestEstimatorLSx_perProb = bestEstimatorPerProb
        
    #---
    
    estimatorLSx = copy.deepcopy(self.estimatorLSx)
    
    estimator = clone(estimatorLSx.estimator)
    estimator.set_params(**paramsEstimatorBest)
    estimator.fit(X = X, y = y)
    
    estimatorLSx.set_params(**paramsLSxBest,
                            estimator = estimator)
    estimatorLSx.fit(X = X, y = y)

    self.bestEstimatorLSx = estimatorLSx

In [ ]:
# show_doc(CrossValidationLSx_combined.fit)

#### Scores for Single Fold

In [ ]:
#| export

# This function evaluates the newsvendor performance for different bin sizes for one specific fold.
# The considered bin sizes

def getFoldScore_combined(estimatorLSx, parameterGridLSx, parameterGridEstimator, cvFold, probs, X, y):
    
    indicesTrain = cvFold[0]
    indicesTest = cvFold[1]
    
    yTrainFold = y[indicesTrain]
    XTrainFold = X[indicesTrain]
    
    yTestFold = y[indicesTest]
    XTestFold = X[indicesTest]
    
    costsDfList = list()
    
    for paramsEstimator in parameterGridEstimator:
        
        estimatorLSx.refitPointEstimator(X = XTrainFold, 
                                         y = yTrainFold,
                                         **paramsEstimator)

        #---

        SAA_fold = SampleAverageApproximation()
        SAA_fold.fit(y = yTrainFold)

        # By setting 'X = None', the SAA results are only computed for a single observation (they are independent of X anyway).
        # In order to receive the final dataframe of SAA results, we simply duplicate this single row as many times as needed.
        quantilesDictSAAOneOb = SAA_fold.predict(X = None, probs = probs, outputAsDf = False)
        quantilesDictSAA = {prob: np.repeat(quantile, len(XTestFold)) for prob, quantile in quantilesDictSAAOneOb.items()}

        #---

        costsPerParamLSx = defaultdict(dict)

        for paramsLSx in parameterGridLSx:

            estimatorLSx.set_params(**paramsLSx)

            estimatorLSx.fit(X = XTrainFold,
                             y = yTrainFold)

            quantilesDict = estimatorLSx.predict(X = XTestFold,
                                                 probs = probs,
                                                 outputAsDf = False)

            #---

            costsDict = dict()

            for prob in probs:            
                costsDict[prob] = getCostRatio(decisions = quantilesDict[prob], 
                                               decisionsSAA = quantilesDictSAA[prob], 
                                               yTest = yTestFold, 
                                               prob = prob)
            
           
            costsPerParamLSx[tuple(paramsLSx.values())] = costsDict

        #---
        
        costsDf = pd.DataFrame.from_dict(costsPerParamLSx, orient = 'index')
        
        paramsLSxNames = list(paramsLSx.keys())
        costsDf.index.names = paramsLSxNames

        costsDf = costsDf.reset_index(drop = False)
        for paramName, value in paramsEstimator.items():
            costsDf[paramName] = value

        paramNames = paramsLSxNames + list(paramsEstimator.keys())
        costsDf = costsDf.set_index(paramNames)
        
        costsDfList.append(costsDf)
        
    #---
    
    costsDf = pd.concat(costsDfList, axis = 0)
    
    return costsDf

## Helper Functions

### Get Cost Ratio

In [ ]:
#| export

def getCostRatio(decisions, decisionsSAA, yTest, prob):

    # Newsvendor Costs of our model
    cost = np.array([prob * (yTest[i] - decisions[i]) if yTest[i] > decisions[i] 
                     else (1 - prob) * (decisions[i] - yTest[i]) 
                     for i in range(len(yTest))]).sum()
    
    # Newsvendor Costs of SAA
    costSAA = np.array([prob * (yTest[i] - decisionsSAA[i]) if yTest[i] > decisionsSAA[i] 
                        else (1 - prob) * (decisionsSAA[i] - yTest[i]) 
                        for i in range(len(yTest))]).sum()
    
    #---
    
    # We have to capture the special case of costSAA == 0, because then we can't compute the 
    # Cost-Ratio using the actual definition.
    if costSAA > 0:
        costRatio = cost / costSAA
    else:
        if cost == 0:
            costRatio = 0
        else:
            costRatio = 1
    
    return costRatio

### Grouped Time Series Split

In [ ]:
#| export

# This function creates the cross-validation folds for every time series. Usually you'd want all test-observations 
# of each fold to refer to the same time period, but this is impossible to ensure in the case of the two-step models,
# because the regression of the non-zero observations will always contain data of different time points. For that
# reason, we refrain from trying to ensure this consistency.
# Instead we organize our splits such that we always move a fixed amount of observations into the future from split
# to split for every time series. This fixed amount of observations is currently set to the test length of the
# corresponding time series.

# In case this function is supposed to be used in the two-step case, data simply has to be filtered before hand
# to only contain positive demand observations.

def groupedTimeSeriesSplit(data, kFolds, testLength, groupFeature, timeFeature):
    
    # We reset the index because we have to access 'group.index' later on and
    # want to make sure that we return the implicit numerical indices for our splits.
    data = data.reset_index(drop = True)

    dataGrouped = data.groupby(groupFeature)
    splitNumbers = np.flip(np.array(range(kFolds)))
    
    foldsList = list()

    for i in splitNumbers:

        trainIndicesList = list()
        valIndicesList = list()

        valIndicesDict = dict()

        for name, group in dataGrouped:

            timeMin = int(group[timeFeature].min())
            timeMax = int(group[timeFeature].max())

            validationTimeMax = timeMax - i * testLength
            trainTimeMax = validationTimeMax - testLength

            trainTimesGroup = np.array(range(timeMin, trainTimeMax + 1))
            valTimesGroup = np.array(range(trainTimeMax + 1, validationTimeMax + 1))

            trainIndicesCheck = [timePoint in trainTimesGroup for timePoint in group[timeFeature]]
            valIndicesCheck = [timePoint in valTimesGroup for timePoint in group[timeFeature]]

            trainIndicesGroup = group.index[trainIndicesCheck]
            valIndicesGroup = group.index[valIndicesCheck]

            trainIndicesList.append(trainIndicesGroup)
            valIndicesList.append(valIndicesGroup)

        trainIndices = np.concatenate(trainIndicesList)
        valIndices = np.concatenate(valIndicesList)
        fold = (trainIndices, valIndices)

        foldsList.append(fold)

    return foldsList

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

# Test Code

In [ ]:
#| hide

# import ipdb
# from lightgbm import LGBMRegressor

# from dddex.loadData import *
# from dddex.wSAA import RandomForestWSAA

In [ ]:
#| hide

# data, XTrain, yTrain, XTest, yTest = loadDataYaz(testDays = 14, 
#                                                  returnXY = True,
#                                                  daysToCut = 0)

In [ ]:
#| hide

# LGBM = LGBMRegressor(boosting_type = 'gbdt',
#                      n_jobs = 1)

# LGBM.fit(X = XTrain, y = yTrain)

In [ ]:
#| hide

# LSKDEx = LevelSetKDEx(estimator = LGBM, binSize = 100, weightsByDistance = False)
# LSKDEx.fit(XTrain, yTrain)

In [ ]:
#| hide

# dataTrain = data[data.label == 'train']

# cvFolds = groupedTimeSeriesSplit(data = dataTrain, 
#                                  kFolds = 3, 
#                                  testLength = 28, 
#                                  groupFeature = 'id', 
#                                  timeFeature = 'dayIndex')

# CV = CrossValidationLSx_combined(estimatorLSx = LSKDEx,
#                                  cvFolds = cvFolds,
#                                  parameterGridLSx = {'binSize': [10, 100, 400],
#                                                     'weightsByDistance': [True, False]},
#                                  parameterGridEstimator = {'max_depth': [3, 4],
#                                                            'n_estimators': [150, 210]},
#                                  probs = [0.01, 0.49, 0.5, 0.999],
#                                  refitPerProb = True)

# CV.fit(X = XTrain, y = yTrain)

# CV2 = QuantileCrossValidation(quantileEstimator = LSKDEx,
#                               cvFolds = cvFolds,
#                               parameterGrid = {'binSize': [10, 100, 400],
#                                                'weightsByDistance': [True, False]},
#                               probs = [0.01, 0.49, 0.5, 0.999],
#                               refitPerProb = True)

# CV2.fit(X = XTrain, y = yTrain)

In [ ]:
# RF = RandomForestWSAA()

# CV = QuantileCrossValidation(quantileEstimator = RF,
#                              cvFolds = cvFolds,
#                              parameterGrid = {'max_depth': [3, 4],
#                                               'n_estimators': [150, 210]},
#                              probs = [0.01, 0.49, 0.5, 0.999],
#                              refitPerProb = True)

# CV.fit(X = XTrain, y = yTrain)